In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
import warnings
import unidecode


warnings.filterwarnings('ignore')
import time, sys

def timeit(method):
    """
    Decorator: Compute the execution time of a function
    :param method: the function
    :return: the method runtime
    """

    def timed(*arguments, **kw):
        ts = time.time()
        result = method(*arguments, **kw)
        te = time.time()

        sys.stdout.write('Time:  %r %2.2f sec\n' % (method.__name__.strip("_"), te - ts))
        sys.stdout.write('------------------------------------\n')
        sys.stdout.flush()
        return result

    return timed

In [16]:
def remove_non_ascii(text):

    return ''.join(i for i in text if ord(i)<128)

In [17]:
wrong_list = []

In [18]:
driver = webdriver.Chrome('/Users/federico/Documents/datascience/jupyter/chromedriver')

In [19]:
@timeit
def get_games(games_url):
    game_list = []
    
    driver.get(games_url)
    games = driver.find_elements_by_xpath('//td[@class="name"]')
    for p in range(len(games)):
        game_list.append(games[p].text.split('\n')[0])

    return game_list

In [20]:
@timeit
def get_gametable_data(system_url, game_list):
    dev_list = []
    gen_list = []
    ser_list = []

    for game in game_list:
        driver.get(system_url)
        try:
            driver.find_element_by_link_text(game).click()
            gametable = driver.find_elements_by_xpath('//table[@class="gameDataTable"]')

            st = gametable[0].text.split('\n')
            try:
                dev_list.append(' '.join(st[1].split()[1:]))
            except IndexError:
                dev_list.append(None)
            try:
                gen_list.append(' '.join(st[2].split()[1:]))
            except IndexError:
                gen_list.append(None)
            try:
                ser_list.append(' '.join(st[3].split()[1:]))
            except IndexError:
                ser_list.append(None)
        except NoSuchElementException:
            dev_list.append(None)
            gen_list.append(None)
            ser_list.append(None)
    return dev_list, gen_list, ser_list
    

In [21]:
@timeit
def get_releasetable_data(system_url, game_list, region):
    jap_name = []
    jap_date = []
    us_name = []
    us_date = []
    eu_name = []
    eu_date = []


    for game in game_list:
        if game == 'Balloon Kid':
            jap_name.append(None)
            jap_date.append(None)
            us_name.append('Balloon Kid')
            us_date.append('Oct 1990')
            eu_name.append('Balloon Kid')
            eu_date.append('1991')
            continue
        if game == 'Balloon Fight GB':
            jap_name.append('Balloon Fight GB')
            jap_date.append('31 Jul 2000')
            us_name.append(None)
            us_date.append(None)
            eu_name.append(None)
            eu_date.append(None)
            continue
            
        driver.get(system_url)
        try:
            driver.find_element_by_link_text(game).click()
            release_table = driver.find_elements_by_xpath('//table[@class="gameRelease"]')[0]

            release_table = release_table.text.split('\n')[1:]
            release_flags = driver.find_elements_by_xpath('//td[@class="flag"]')
            if len(release_flags) > 3:
                release_flags = release_flags[:3]

            countries = []

            for flag in release_flags:
                country = flag.find_element_by_tag_name('img').get_attribute('src')[-6:-4]
                countries.append(country)

                
            if len(countries) == 1:
                if countries[0] == 'jp':
                    if len(release_table) % 2 != 0:
                        jap_name.append(release_table[1])
                        jap_date.append(release_table[2])
                        us_name.append(None)
                        us_date.append(None)
                        eu_name.append(None)
                        eu_date.append(None)
                    else: #casi F-Zero
                        jap_name.append(release_table[0])
                        jap_date.append(release_table[1])
                        us_name.append(None)
                        us_date.append(None)
                        eu_name.append(None)
                        eu_date.append(None)                        

                elif countries[0] == 'us':
                    jap_name.append(None)
                    jap_date.append(None)
                    us_name.append(release_table[0])
                    us_date.append(release_table[1])
                    eu_name.append(None)
                    eu_date.append(None)
                elif countries[0] == 'eu':
                    jap_name.append(None)
                    jap_date.append(None)
                    us_name.append(None)
                    us_date.append(None)
                    eu_name.append(release_table[0])
                    eu_date.append(release_table[1])


            elif len(countries) == 2:
                if len(release_table) % 2 != 0:
                    if countries[0] == 'jp':
                        jap_name.append(release_table[1])
                        jap_date.append(release_table[2])
                        if countries[1] == 'us':
                            us_name.append(release_table[3])
                            us_date.append(release_table[4])
                            eu_name.append(None)
                            eu_date.append(None)
                        else:
                            eu_name.append(release_table[3])
                            eu_date.append(release_table[4])
                            us_name.append(None)
                            us_date.append(None)
                else: #casi F-Zero
                    if countries[0] == 'jp':
                        jap_name.append(release_table[0])
                        jap_date.append(release_table[1])
                        if countries[1] == 'us':
                            us_name.append(release_table[2])
                            us_date.append(release_table[3])
                            eu_name.append(None)
                            eu_date.append(None)
                        else:
                            eu_name.append(release_table[2])
                            eu_date.append(release_table[3])
                            us_name.append(None)
                            us_date.append(None)                    


                if countries[0] == 'us':
                    us_name.append(release_table[0])
                    us_date.append(release_table[1])

                    if countries[1] == 'jp':
                        if len(release_table) % 2 != 0:                        
                            jap_name.append(release_table[3])

                            jap_date.append(release_table[4])
                            eu_name.append(None)
                            eu_date.append(None)
                        else:
                            jap_name.append(release_table[2])

                            jap_date.append(release_table[3])
                            eu_name.append(None)
                            eu_date.append(None)                            
                    else:
                        eu_name.append(release_table[2])
                        eu_date.append(release_table[3])
                        jap_name.append(None)
                        jap_date.append(None)

                if countries[0] == 'eu':
                    eu_name.append(release_table[0])
                    eu_date.append(release_table[1])
                    if countries[1] == 'jp':
                        if len(release_table) % 2 != 0:                        
                            jap_name.append(release_table[3])

                            jap_date.append(release_table[4])
                            eu_name.append(None)
                            eu_date.append(None)
                        else:
                            jap_name.append(release_table[2])

                            jap_date.append(release_table[3])
                            eu_name.append(None)
                            eu_date.append(None)                
                    else:
                        us_name.append(release_table[2])
                        us_date.append(release_table[3])
                        jap_name.append(None)
                        jap_date.append(None)


            elif len(countries) == 3:
                if len(release_table) % 2 != 0:                        
                    if countries[0] == 'jp':
                        jap_name.append(release_table[1])
                        jap_date.append(release_table[2])

                        if countries[1] == 'us':
                            us_name.append(release_table[3])
                            us_date.append(release_table[4])
                            eu_name.append(release_table[5])
                            eu_date.append(release_table[6])
                        else:
                            eu_name.append(release_table[3])
                            eu_date.append(release_table[4])
                            us_name.append(release_table[5])

                            us_date.append(release_table[6])


                    elif countries[0] == 'us':
                        us_name.append(release_table[0])

                        us_date.append(release_table[1])

                        if countries[1] == 'jp':
                            jap_name.append(release_table[3])
                            jap_date.append(release_table[4])
                            eu_name.append(release_table[5])
                            eu_date.append(release_table[6])

                        else:
                            eu_name.append(release_table[2])

                            eu_date.append(release_table[3])

                            jap_name.append(release_table[5])

                            jap_date.append(release_table[6])


                    elif countries[0] == 'eu':
                        eu_name.append(release_table[0])

                        eu_date.append(release_table[1])

                        if countries[1] == 'jp':
                            jap_name.append(release_table[3])

                            jap_date.append(release_table[4])
                            us_name.append(release_table[5])

                            us_date.append(release_table[6])

                        else:
                            us_name.append(release_table[2])
                            us_date.append(release_table[3])
                            jap_name.append(release_table[5])
                            jap_date.append(release_table[6])
                            
                else: #casi F-Zero                        
                    if countries[0] == 'jp':
                        jap_name.append(release_table[0])
                        jap_date.append(release_table[1])

                        if countries[1] == 'us':
                            us_name.append(release_table[2])
                            us_date.append(release_table[3])
                            eu_name.append(release_table[4])
                            eu_date.append(release_table[5])
                        else:
                            eu_name.append(release_table[2])
                            eu_date.append(release_table[3])
                            us_name.append(release_table[4])

                            us_date.append(release_table[5])


                    elif countries[0] == 'us':
                        us_name.append(release_table[0])

                        us_date.append(release_table[1])

                        if countries[1] == 'jp':
                            jap_name.append(release_table[2])
                            jap_date.append(release_table[3])
                            eu_name.append(release_table[4])
                            eu_date.append(release_table[5])

                        else:
                            eu_name.append(release_table[2])

                            eu_date.append(release_table[3])

                            jap_name.append(release_table[4])

                            jap_date.append(release_table[5])


                    elif countries[0] == 'eu':
                        eu_name.append(release_table[0])

                        eu_date.append(release_table[1])

                        if countries[1] == 'jp':
                            jap_name.append(release_table[2])

                            jap_date.append(release_table[3])
                            us_name.append(release_table[4])

                            us_date.append(release_table[5])

                        else:
                            us_name.append(release_table[2])
                            us_date.append(release_table[3])
                            jap_name.append(release_table[4])
                            jap_date.append(release_table[6])

###
                            
        except (IndexError, NoSuchElementException):
            
            wrong_list.append(game)
        
            if region == 'jp':      
                jap_name.append(game)
                jap_date.append(None)
                us_name.append(None)
                us_date.append(None)
                eu_name.append(None)
                eu_date.append(None)
                print("Something wrong with game:", game)
            elif region == 'us':
                jap_name.append(None)
                jap_date.append(None)
                us_name.append(game)
                us_date.append(None)
                eu_name.append(None)
                eu_date.append(None)
                print("Something wrong with game:", game)
            elif region == 'eu':
                jap_name.append(None)
                jap_date.append(None)
                us_name.append(None)
                us_date.append(None)
                eu_name.append(game)
                eu_date.append(None)
                print("Something wrong with game:", game)
            
                
    return jap_name, jap_date, us_name, us_date, eu_name, eu_date

In [22]:
#@timeit

def getDev (game, df, w):
    roles = ['Executive', 'Sub', 'Design', 'Level', 'Programming', 'Assistant', 'Associate', 
             'Program', 'General', 'Chief']
    producer_flag = False
    director_flag = False
    
    title = remove_non_ascii(game)
            
    if title.startswith("the-"):
        title = title[4:]
                              
    print ("Mobygames:", title)
    driver.get("https://www.mobygames.com/game/gameboy/" + title + "/credits")
    devs = driver.find_elements_by_xpath('//tr[@class="crln"]')
    for d in devs:
        
        if producer_flag == False:
            role_check = True
            if 'Producer' in d.text:
                for role in roles:
                    if role in d.text:
                        role_check = False
                if role_check == True:
                    df['Producer'][w] = str(d.text).replace('Producers', '').replace('Producer', "").replace('Lead', "")
                    producer_flag = True
                    
        if director_flag == False:
            role_check = True
            if 'Director' in d.text:
                for role in roles:
                    if role in d.text:
                        role_check = False
                if role_check == True:
                    df['Director'][w] = str(d.text).replace('Game', '').replace('Directors', '').replace('Director', "").replace('Lead', "")
                    director_flag = True
                    
        if producer_flag == True and director_flag == True:
            break
    
    if producer_flag == False:
        driver.get("http://kyoto-report.wikidot.com/" + game)

        devs = driver.find_elements_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div/div[3]/table[2]/tbody/tr/td[2]/p[1]')
        
        if len(devs) > 0:
            producer_list = str(devs[0].text).split('\n⌂ ')
            producer_value = ""
            for k in range(1,len(producer_list)-1):
                producer_value = producer_value + producer_list[k].split('\n')[0] + ", "
            producer_value = producer_value + producer_list[len(producer_list)-1].split('\n')[0] 
            df['Producer'][w] = producer_value
            
            if len(producer_list) == 2:
                producer_value = producer_list[1].split('\n')[0]
            producer_flag = True
    
    if producer_flag == False:
        driver.get("http://kyoto-report.wikidot.com/" + game + "-credits")
        devs = driver.find_elements_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div/div[3]/p[2]')
        
        if len(devs) > 0:
            producer_list = devs[0].text.split('\n⌂ ')
            producer_value = ""

            for k in range(1,len(producer_list)-1):
                producer_value = producer_value + producer_list[k].split('\n')[0] + ", "
                producer_value = producer_value + producer_list[len(producer_list)-1].split('\n')[0] 
                
            if len(producer_list) == 2:
                producer_value = producer_list[1].split('\n')[0]
            
            df['Producer'][w] = producer_value
            producer_flag = True

    if director_flag == False:
        driver.get("http://kyoto-report.wikidot.com/" + game)

        devs = driver.find_elements_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div/div[3]/table[2]/tbody/tr/td[1]/p[1]')
        
        if len(devs) > 0:

            director_list = str(devs[0].text).split('\n⌂ ')
            director_value = ""
            for j in range(1,len(director_list)-1):
                director_value = director_value + director_list[j].split('\n')[0] + ", "
            director_value = director_value + director_list[len(director_list)-1].split('\n')[0] 
            if len(director_list) == 2:
                director_value = director_list[1].split('\n')[0]
            df['Director'][w] = director_value

            director_flag = True
        
    if director_flag == False:
        director_list = []
        director_value = ""

        driver.get("http://kyoto-report.wikidot.com/" + game + "-credits")
        devs = driver.find_elements_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div/div[3]/p[3]')
       
        if len(devs) > 0:
            director_list = devs[0].text.split('\n⌂ ')
            for k in range(1,len(director_list)-1):
                director_value = director_value + director_list[k].split('\n')[0] + ", "
                director_value = director_value + director_list[len(director_list)-1].split('\n')[0]   
                if len(director_list) == 2:
                    director_value = director_list[1].split('\n')[0]                
                df['Director'][w] = director_value
                
                director_flag = True


    return df

In [23]:
@timeit
def main(scrapeDev=False):
    jp_url = 'https://nindb.net/gb/games-jp.html' #Japan
    pw_url = "https://nindb.net/gb/np-jp.html" #Additional Japanese games (Nintendo Power)
    us_url = 'https://nindb.net/gb/games-us.html' #North America
    eu_url = 'https://nindb.net/gb/games-eu.html' #Europe
    

    
    print('Getting game titles...')
    game_list = get_games(jp_url)
    game_list_pw = [g for g in get_games(pw_url) if g not in game_list] #Nintendo Power
    
    #game_list_us = [g for g in get_games(us_url) if g not in game_list + game_list_pw]
    
    #game_list_eu = [] #[g for g in get_games(eu_url) if g not in game_list + game_list_pw + game_list_us]
    
    
    print('Getting release data...')
    jap_name, jap_date, us_name, us_date, eu_name, eu_date = get_releasetable_data(jp_url, game_list, 'jp')
    jap_name_pw, jap_date_pw, us_name_pw, us_date_pw, eu_name_pw, eu_date_pw = get_releasetable_data(pw_url, game_list_pw, 'jp')
    #jap_name_us, jap_date_us, us_name_us, us_date_us, eu_name_us, eu_date_us = get_releasetable_data(us_url, game_list_us, 'usa')
    #jap_name_eu, jap_date_eu, us_name_eu, us_date_eu, eu_name_eu, eu_date_eu = get_releasetable_data(eu_url, game_list_eu, 'eu')
    
    
    
    print('Getting game data...')
    dev_list, gen_list, ser_list = get_gametable_data(jp_url, game_list)
    dev_list_pw, gen_list_pw, ser_list_pw = get_gametable_data(pw_url, game_list_pw)
    #dev_list_us, gen_list_us, ser_list_us = get_gametable_data(us_url, game_list_us)
    #dev_list_eu, gen_list_eu, ser_list_eu = get_gametable_data(eu_url, game_list_eu)

    
    game_list_us = [g for g in get_games(us_url) if g not in us_name]
    jap_name_us, jap_date_us, us_name_us, us_date_us, eu_name_us, eu_date_us = get_releasetable_data(us_url, game_list_us, 'us')
    dev_list_us, gen_list_us, ser_list_us = get_gametable_data(us_url, game_list_us)
    
    
    game_list_eu = [g for g in get_games(eu_url) if g not in set(eu_name + eu_name_us)]
    jap_name_eu, jap_date_eu, us_name_eu, us_date_eu, eu_name_eu, eu_date_eu = get_releasetable_data(eu_url, game_list_eu, 'eu')
    dev_list_eu, gen_list_eu, ser_list_eu = get_gametable_data(eu_url, game_list_eu)
    

    #try:
    print('Building dataframe...')
    df = pd.DataFrame()
    df['JP Title'] = jap_name + jap_name_pw + jap_name_us + jap_name_eu
    df['JP Release Date'] = jap_date + jap_date_pw + jap_date_us + jap_date_eu
    df['US Title'] = us_name + us_name_pw + us_name_us + us_name_eu
    df['US Release Date'] = us_date + us_date_pw + us_date_us + us_date_eu
    df['EU Title'] = eu_name + eu_name_pw + eu_name_us + eu_name_eu
    df['EU Release Date'] = eu_date + eu_date_pw + eu_date_us + eu_date_eu

    df['System'] = ["GB"] * len(df)
    df['Developer'] = dev_list + dev_list_pw + dev_list_us + dev_list_eu
    df['Genre'] = gen_list + gen_list_pw + gen_list_us + gen_list_eu
    df['Series'] = ser_list + ser_list_pw + ser_list_us + ser_list_eu
    #except:
        #return game_list + game_list_pw + jap_name_us + jap_name_eu, us_name + us_name_pw + us_name_us + us_name_eu
        
    
    df = df.drop_duplicates().reset_index(drop=True)
    
    
    if scrapeDev == True:
        print('Getting developers data...')

        df['Producer'] = [None] * len(df)
        df['Director'] = [None] * len(df)
        for w in range(len(df)):
            if df['JP Title'][w] == 'Mother':
                df['US Title'][w] = "EarthBound Beginnings"
                df['EU Title'][w] = df['US Title'][w]
            if df['JP Title'][w] == 'Yume Koujou: Dokidoki Panic':
                df['JP Title'][w] == 'Yume Kōjō: Dokidoki Panic'


            if df['US Title'][w] is not None:
                title = (df['US Title'][w]).lower().replace(' & ', '-').replace(' ', '-').replace(':-', '-').replace("'", '').replace("ū", '').replace(".", '').replace("!", '').replace("ō", '')
                if title == 'donkey-kong':
                    title = 'donkey-kong-'
                elif title == 'battleclash':
                    title = 'battle-clash'
                df = getDev (title, df, w)
            elif df['EU Title'][w] is not None:
                title = (df['EU Title'][w]).lower().replace(' & ', '-').replace(' ', '-').replace(':-', '-').replace("'", '').replace("ū", '').replace(".", '').replace("!", '').replace("ō", '')
                df = getDev (title, df, w)
            elif df['JP Title'][w] is not None:
                title = (df['JP Title'][w]).lower().replace(' & ', '-').replace(' ', '-').replace(':-', '-').replace("'", '').replace("ū", '').replace(".", '').replace("!", '').replace("ō", '')

                df = getDev (title, df, w)
    
    
    print('Done!')
    return df


In [24]:
df  = main(True) 
#True = Prende i dev. Default = False
#Il drop_duplicate lo fa di default prima di prendere i dev
#Fa tutte le regioni

#SIDE EFFECT:
#1. I giochi rilasciati "ad episodi" su Disk System appaiono una volta sola, con due date di uscita
#2. I giochi nella Wrong List rilasciati in più regioni appaiono una volta per regione, con le altre regioni "None"


Getting game titles...
Time:  'get_games' 3.53 sec
------------------------------------
Time:  'get_games' 0.43 sec
------------------------------------
Getting release data...
Time:  'get_releasetable_data' 20.91 sec
------------------------------------
Time:  'get_releasetable_data' 2.06 sec
------------------------------------
Getting game data...
Time:  'get_gametable_data' 4.44 sec
------------------------------------
Time:  'get_gametable_data' 1.50 sec
------------------------------------
Time:  'get_games' 0.55 sec
------------------------------------
Something wrong with game: Nintendo World Cup
Something wrong with game: Super R.C. Pro-Am
Something wrong with game: Space Invaders
Something wrong with game: Arcade Classic #1. Asteroids & Missile Command
Something wrong with game: Arcade Classic #2. Centipede & Millipede
Something wrong with game: Arcade Classic #3. Galaga & Galaxian
Something wrong with game: Street Fighter II
Something wrong with game: Arcade Classic #4. Defe

In [28]:
df[:50]

,JP Title,JP Release Date,US Title,US Release Date,EU Title,EU Release Date,System,Developer,Genre,Series,Producer,Director
0,Alleyway,21 Apr 1989,Alleyway,Aug 1989,Alleyway,Sep 1990,GB,Nintendo,Action / Puzzle,None,Gunpei Yokoi,None
1,Baseball,21 Apr 1989,Baseball,Aug 1989,Baseball,Sep 1990,GB,Nintendo,Sport (Baseball),Baseball Series,None,None
2,Super Mario Land,21 Apr 1989,Super Mario Land,Aug 1989,Super Mario Land,28 Sep 1990,GB,Nintendo,2D Platformer,Super Mario Series,Gunpei Yokoi,Satoru Okada
3,Yakuman,21 Apr 1989,None,None,None,None,GB,Nintendo,Mahjong,Mahjong Series,None,None
4,Tennis,29 May 1989,Tennis,Aug 1989,Tennis,Sep 1990,GB,Nintendo,Sport (Tennis),Tennis Series,None,None
5,Tetris,14 Jun 1989,Tetris,Aug 1989,Tetris,Sep 1990,GB,Nintendo,Puzzle,Tetris Series,None,None
6,Golf,28 Nov 1989,Golf,Feb 1990,Golf,Sep 1990,GB,Nintendo,Sport (Golf),Golf Series,Masayuki Uemura,None
7,Solar Striker,26 Jan 1990,Solar Striker,Feb 1990,Solar Striker,Sep 1990,GB,Nintendo,Shooting,None,Gunpei Yokoi,"Satoru Okada, Keisuke Terasaki"
8,Qix,13 Apr 1990,Qix,May 1990,Qix,Sep 1990,GB,"Nintendo, Taito",Action / Puzzle,None,None,None
9,Dr. Mario,27 Jul 1990,Dr. Mario,Dec 1990,Dr. Mario,1991,GB,Nintendo,Puzzle,Super Mario Series,Gunpei Yokoi,None


In [14]:
set(wrong_list)

{'Adventures of Lolo',
 'Arcade Classic #1. Asteroids & Missile Command',
 'Arcade Classic #2. Centipede & Millipede',
 'Arcade Classic #3. Galaga & Galaxian',
 'Arcade Classic #4. Defender & Joust',
 'Battle Arena Toshinden',
 'Bomberman GB',
 'Boulder Dash',
 'Burai Fighter Deluxe',
 'Double Dragon',
 'Dynablaster',
 'Fortress of Fear: Wizards & Warriors X',
 "Gargoyle's Quest",
 'King of the Zoo',
 'Kung-Fu Master',
 'Kwirk',
 'Magnetic Soccer',
 'Mega Man II',
 "Mega Man: Dr. Wily's Revenge",
 'Mercenary Force',
 'Ms. Pac-Man',
 "Nigel Mansell's World Championship Racing",
 'Nintendo World Cup',
 'Othello',
 "Pinball: Revenge of the 'Gator",
 'Pocket Bomberman',
 'R-Type',
 'Side Pocket',
 'Space Invaders',
 'Street Fighter II',
 'Super R.C. Pro-Am',
 'Tetris Blast',
 'Tetris Plus',
 'The Amazing Spider-Man',
 'The Chessmaster',
 "The King of Fighters '95",
 'The Rescue of Princess Blobette',
 'Vegas Stakes'}

In [31]:
game = 'balloon-fight-gb'

roles = ['Executive', 'Sub', 'Design', 'Level', 'Programming', 'Assistant', 'Associate', 
         'Program', 'General', 'Chief']
producer_flag = False
director_flag = False

title = remove_non_ascii(game)

if title.startswith("the-"):
    title = title[4:]

driver.get("https://www.mobygames.com/game/gameboy/" + title + "/credits")
devs = driver.find_elements_by_xpath('//tr[@class="crln"]')
for d in devs:

    if producer_flag == False:
        role_check = True
        if 'Producer' in d.text:
            for role in roles:
                if role in d.text:
                    role_check = False
            if role_check == True:
                print ("Producer", str(d.text).replace('Producers', '').replace('Producer', "").replace('Lead', ""))
                producer_flag = True

    if director_flag == False:
        role_check = True
        if 'Director' in d.text:
            for role in roles:
                if role in d.text:
                    role_check = False
            if role_check == True:
                print ("Director", str(d.text).replace('Producers', '').replace('Producer', "").replace('Lead', ""))
                director_flag = True

    if producer_flag == True and director_flag == True:
        break